In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# AUGUSTUS is a piece of software that predicts genes ab initio using Hidden Markov Models.
file_path_augustus = os.path.join('..', 'input', 'genes-augustus.csv')
augustus_df = pd.read_csv(file_path_augustus)

augustus_df.head()

In [ ]:
augustus_df.info()

txStart: transcription start site usually includes UTR(untranslated regions - doesn't contibute to protein sequence)

cdsStart: downstream to the txStart, start of coding sequence - this is what ACTUALLY get translated to protein

exonCount: mRNA fragments that are joined together during splicing to form full length CDS

**Lets find out what is the average length of 5' UTRs**. 

***5'UTR = txStart - cdsStart***

In [ ]:
UTR_5_df = abs(augustus_df['txStart'] - augustus_df['cdsStart'])
UTR_5_df.head()

In [ ]:
sns.set_style('whitegrid')
sns.set_context('talk')
sns.set_palette('husl')
g = sns.boxenplot(data=UTR_5_df)

In [ ]:
g = sns.boxenplot(data=UTR_5_df)
g.set_yscale('log')

**Lets check the exon count**

In [ ]:
sns.set_palette('bright')
g = sns.boxenplot(data=augustus_df['exonCount'])

Hmm  there are a few outliers with more than 20 exons

Lets check if there is a correlation between mRNA length and exon number

mRNA length = abs(txEnd - txStart)

In [ ]:
mRNA_df = abs(augustus_df['txEnd'] - augustus_df['txStart'])
normalization_factor = mRNA_df.max()/augustus_df['exonCount'].max()
mRNA_df = mRNA_df / mRNA_df.max()
exon_num_df = augustus_df['exonCount']*normalization_factor
exon_num_df = exon_num_df/exon_num_df.max()

In [ ]:

corr = mRNA_df.corr(exon_num_df)
print(corr)


In [ ]:
exon_mRNA_df = pd.concat([exon_num_df, mRNA_df], axis=1)
exon_mRNA_df.columns = ['exonCount', 'mRNA_len']
exon_mRNA_df.head()

In [ ]:
import matplotlib.pyplot as plt
sns.set_palette('bright')
ax = sns.scatterplot(x='mRNA_len', y='exonCount', data=exon_mRNA_df)


In [ ]:
augustus_df['exonCount'].describe()

There exists a strong correlation upto certain mRNA length. After which the effect of mRNA length on exon number is minimal.

It is likely that there was some slection pressure against indefinite increase in exon number